In [1]:
ls

README.md                     未命名.ipynb
SPGC-metadata-2018-07-18.csv


In [93]:
import pandas as pd
from collections import defaultdict

import requests
from bs4 import BeautifulSoup 

import sys

# flush print
def flushPrint(d):
    sys.stdout.write('\r')
    sys.stdout.write(str(d))
    sys.stdout.flush()


In [4]:
# https://zenodo.org/record/2422561#.YSckv9MzbOQ
df = pd.read_csv('SPGC-metadata-2018-07-18.csv')
df[:3]

,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type
0,PG0,NaN,NaN,NaN,NaN,NaN,NaN,set(),Text
1,PG1,The Declaration of Independence of the United ...,"Jefferson, Thomas",1743.0,1826.0,['en'],604.0,"{'United States -- History -- Revolution, 1775...",Text
2,PG2,The United States Bill of Rights: The Ten Orig...,United States,NaN,NaN,['en'],158.0,"{'Civil rights -- United States -- Sources', '...",Text


In [36]:
df['Fiction'] = [1 if 'Fiction' in i  else 0 for i in df['subjects']]

In [38]:
dff = df[df['Fiction']==1]
len(dff)

13302

In [39]:
dff

,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type,Fiction
15,PG15,Moby Dick,"Melville, Herman",1819.0,1891.0,['en'],905.0,"{'Mentally ill -- Fiction', 'Whaling ships -- ...",Text,1
16,PG16,Peter Pan,"Barrie, J. M. (James Matthew)",1860.0,1937.0,['en'],5359.0,"{'Fantasy literature', 'Pirates -- Fiction', '...",Text,1
24,PG24,O Pioneers!,"Cather, Willa",1873.0,1947.0,['en'],448.0,"{'Swedish Americans -- Fiction', 'Domestic fic...",Text,1
27,PG27,Far from the Madding Crowd,"Hardy, Thomas",1840.0,1928.0,['en'],411.0,"{'Love stories', 'Wessex (England) -- Fiction'...",Text,1
32,PG32,Herland,"Gilman, Charlotte Perkins",1860.0,1935.0,['en'],642.0,"{'Women -- Fiction', 'Utopias -- Fiction', 'Ut...",Text,1
...,...,...,...,...,...,...,...,...,...,...
56561,PG56563,Her Sailor: A Love Story,"Saunders, Marshall",1861.0,1947.0,['en'],13.0,{'New England -- Social life and customs -- Fi...,Text,1
56650,PG56652,Kitty of the Roses,"Barbour, Ralph Henry",1870.0,1944.0,['en'],25.0,"{'Southern States -- Fiction', 'Romance fiction'}",Text,1
56758,PG56760,De drie steden: Lourdes,"Zola, Émile",1840.0,1902.0,['nl'],23.0,"{'Lourdes (France) -- Fiction', 'Catholics -- ...",Text,1
57135,PG57137,"Barry Wynn: Or, The Adventures of a Page Boy i...","Barton, George",1866.0,1940.0,['en'],66.0,{'Capitol pages -- Fiction'},Text,1


# download 13302 Fictions from Gutenberg

In [49]:
base_url = 'https://www.gutenberg.org/ebooks/'
urls = [base_url+i.replace('PG', '')   for i in dff['id']]

In [52]:
print(urls[0])

https://www.gutenberg.org/ebooks/15


In [91]:
urls[0].replace(base_url, '')+'.txt'

'https://www.gutenberg.org/ebooks/15'

In [92]:
error_log=[]
for k, i in enumerate(urls[:5]):
    flushPrint(k, i)
    try:
        content = requests.get(i)
        soup = BeautifulSoup(content.text, 'html.parser') 
        list_href=[i['href'] for i in soup.find_all('a', {'class', 'link'})]
        txt_href = [i for i in list_href  if '.txt' in i][0]
        content = requests.get('https://www.gutenberg.org'+txt_href)
        content.encoding = 'utf-8'
        with open(i.replace(base_url, '')+'.txt', 'w') as f:
            f.write(content.text)
    except Exception as e:
        error_log.append([i, e])
        pass
        

0


In [56]:
content = requests.get(urls[0])
soup = BeautifulSoup(content.text, 'html.parser') 

In [75]:
list_href=[i['href'] for i in soup.find_all('a', {'class', 'link'})]
txt_href = [i for i in list_href  if '.txt' in i][0]

In [83]:
content = requests.get('https://www.gutenberg.org'+txt_href)
content.encoding = 'utf-8'

In [86]:
txt_href.replace('/', '|')

'/files/15/15-0.txt'

In [87]:
with open(txt_href.replace('/', '|'), 'w') as f:
    f.write(content.text)

In [12]:
cat_dic = defaultdict(int)
for i in df.index:
    catset = eval(df['subjects'][i])
    for j in catset:
        cat_dic[j] += 1

In [31]:
df_cat = pd.DataFrame(cat_dic.items(), columns = ['cat', 'freq'])

In [34]:
df_cat.sort_values(by = 'freq', ascending = False)[:50]

,cat,freq
554,Fiction,2007
308,Short stories,1858
93,Science fiction,1652
32,Adventure stories,923
60,Historical fiction,759
66,Love stories,719
155,Conduct of life -- Juvenile fiction,699
390,Poetry,693
436,Detective and mystery stories,606
6985,English wit and humor -- Periodicals,556
